In [34]:
import pandas as pd

In [39]:
albums = pd.read_csv("albums.csv")
albums.drop(["Unnamed: 0"], axis=1, inplace=True)
albums.head()

,spotify_id,youtube_id,project_name,artist,project_type,tracks,project_art,year,rating
0,1B5Lwt0D1ZetRreaCBYobP,GMKfsj1fg5g,all day,girl talk,album,14.0,https://i.scdn.co/image/ab67616d000048510192ca...,2008.0,8.0
1,20r762YmB5HeofjMCiPMLv,Jo4S2qlQGs0,my beautiful dark twisted fantasy,kanye west,album,13.0,https://i.scdn.co/image/ab67616d00004851d9194a...,2010.0,6.0
2,35tQBmq1RblVUzAvfsw5uO,AyzpDXRVbwA,scary monsters and nice sprites,skrillex,ep,9.0,https://i.scdn.co/image/ab67616d0000485181cfaf...,2010.0,7.0
3,0X7WyEKdm5afGj1fmD7Blx,lVj-lS3bfUc,broken bells,broken bells,ep,10.0,https://i.scdn.co/image/ab67616d00004851e198b6...,2010.0,5.0
4,5zi7WsKlIiUXv09tbGLKsE,_2RPGAA6p4E,bastard,"tyler, the creator",album,12.0,https://i.scdn.co/image/ab67616d00004851700588...,2019.0,7.0


In [40]:
albums.drop(["youtube_id", "project_name", "artist", "project_type", "tracks", "project_art", "year"], axis=1, inplace=True)
albums.drop_duplicates(inplace=True)
albums.dropna(inplace=True)

In [41]:
tracks = pd.read_csv("tracks.csv")
tracks.drop(["Unnamed: 0"], axis=1, inplace=True)
tracks.head()

,spotify_id,album_id,youtube_id,name,duration,explicit,preview,key,mode,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,5udnrY00yVUOAzupil2H56,1B5Lwt0D1ZetRreaCBYobP,GMKfsj1fg5g,Play Your Part (Pt. 1),284799.0,1.0,https://p.scdn.co/mp3-preview/eae6854721adddb5...,5.0,0.0,0.0226,0.791,0.675,0.000000,0.2350,-7.447,0.233,0.698,149.980
1,6GJ9BTsYMQc6BZ8QxogDcw,1B5Lwt0D1ZetRreaCBYobP,GMKfsj1fg5g,Shut The Club Down,186719.0,1.0,https://p.scdn.co/mp3-preview/d09ac93dd40658b4...,11.0,0.0,0.0772,0.793,0.693,0.000001,0.0754,-7.589,0.394,0.726,165.073
2,1f9qTGnov1fiaEK451GWjG,1B5Lwt0D1ZetRreaCBYobP,GMKfsj1fg5g,Still Here,237098.0,1.0,https://p.scdn.co/mp3-preview/da072d94e18e0732...,11.0,0.0,0.0899,0.786,0.699,0.000000,0.3790,-8.403,0.293,0.525,90.035
3,4fOFWrYmRUfS9sC9TyOlUb,1B5Lwt0D1ZetRreaCBYobP,GMKfsj1fg5g,What It's All About,254705.0,1.0,https://p.scdn.co/mp3-preview/f96a9d71ad15b68a...,11.0,0.0,0.0651,0.714,0.709,0.000000,0.7430,-7.086,0.281,0.764,90.109
4,4XTl02cmvjPPoQcCDwINvy,1B5Lwt0D1ZetRreaCBYobP,GMKfsj1fg5g,Set It Off,221840.0,1.0,https://p.scdn.co/mp3-preview/2c829841853a94d1...,1.0,1.0,0.0426,0.592,0.786,0.000021,0.3340,-7.132,0.228,0.438,102.520


In [42]:
tracks.drop(["spotify_id", "youtube_id", "name", "preview"], axis=1, inplace=True)
tracks.drop_duplicates(inplace=True)

In [43]:
from pandas.api.types import is_numeric_dtype

print(len(tracks))
for column, _ in tracks.items():
    if not pd.to_numeric(tracks[column], errors='coerce').notnull().all():
        continue
    
    Q1 = tracks[column].quantile(0.25)
    Q3 = tracks[column].quantile(0.75)
    IQR = Q3 - Q1

    tracks = tracks.loc[(tracks[column] > (Q1 - 1.5 * IQR)) & (tracks[column] < (Q3 + 1.5 * IQR))]
print(len(tracks))

29668
20201


In [44]:
print(len(albums))
albums.reset_index()
tracks_copy = tracks.copy()
overall_averages = pd.DataFrame()
for index, row in albums.iterrows():
    album_tracks = tracks_copy.loc[tracks["album_id"] == row["spotify_id"]]

    # remove albums with zero tracks after purging outliers
    if len(album_tracks) == 0:
        albums = albums[albums["spotify_id"] != row["spotify_id"]]

    album_tracks = album_tracks.copy().drop(["album_id"], axis=1)
    album_averages = album_tracks.mean(numeric_only=True)
    overall_averages = pd.concat([overall_averages, pd.DataFrame({k:[v] for k,v in album_averages.items()})], ignore_index=True)
print(len(albums))
print(len(overall_averages))

2455
2239
2455


In [45]:
album_ratings = albums.rating
n = 4 # n is the number of potential ratings, classes
album_ratings = album_ratings.apply(lambda x : round(x / 10 * (n-1)))

album_ratings = album_ratings.map({0:'poor', 1:'fair', 2:'good', 3:'excellent'})

In [46]:
# columns_to_remove = ["acousticness", "danceability", "liveness", "valence"]
columns_to_remove = []
inputs = overall_averages.copy().drop(columns_to_remove, axis=1)
inputs.dropna(inplace=True)
inputs.head()
print(len(inputs))

2239


In [47]:
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

import time
results = {}

for model in [
    GaussianNB(),
    RandomForestClassifier(),
    LinearDiscriminantAnalysis(),
    KNeighborsClassifier(n_neighbors=20),
]:
    total_score = 0
    iters = 10

    start_time = time.perf_counter()
    for i in range(iters):
        x_train, x_test, y_train, y_test = train_test_split(
            inputs, album_ratings, test_size=0.2)

        model.fit(x_train, y_train)
        score = model.score(x_test, y_test)
        total_score += score
    end_time = time.perf_counter()

    classifier = model.__class__.__name__
    results[classifier] = {
        "time": "{:0.4f}s".format(end_time - start_time),
        "average_score": "{:0.3f}%".format(
            total_score / iters * 100),
    }

In [48]:
print(f"{'Algorithm':<30} {'Time Taken':<10} {'Average Score':<10}")
print(f"{'---------':<30} {'----------':<10} {'-------------':<10}")
for k, v in results.items():
    print(f"{k:<30} {v['time']:<10} {v['average_score']:<10}")

Algorithm                      Time Taken Average Score
---------                      ---------- -------------
GaussianNB                     0.0831s    79.844%   
RandomForestClassifier         5.9781s    80.067%   
LinearDiscriminantAnalysis     0.2283s    81.205%   
KNeighborsClassifier           0.3548s    80.179%   


In [71]:
print("RealData:")
print(album_ratings.value_counts().to_string(), "\n")

for model in [
    GaussianNB(),
    RandomForestClassifier(),
    LinearDiscriminantAnalysis(),
    KNeighborsClassifier(n_neighbors=20),
]:
    model.fit(x_train, y_train)
    prediction = model.predict(inputs)

    df = pd.DataFrame({'data':prediction})
    print(f"{model.__class__.__name__}:")
    print(df['data'].value_counts().to_string(), "\n")

RealData:
good         1806
fair          331
excellent      89
poor           13 

GaussianNB:
good    2239 

RandomForestClassifier:
good         1895
fair          263
excellent      69
poor           12 

LinearDiscriminantAnalysis:
good         2235
fair            3
excellent       1 

KNeighborsClassifier:
good    2239 

